In [22]:
import os
import networkx as nx
import shlex
import json
import csv
import nltk
import itertools

In [30]:
data_folder_channels='/media/aruiz/data/channels_clean_data/'
fileName2=os.path.join(data_folder_channels,'minecraft_ch_data_clean.json')
g = nx.Graph()

In [35]:
i=0
with open(fileName2) as f:
        for line in f:
            i+=1
            if (i==4):
                break
            json_dict = json.loads(line)
            all_keys=[]
            if (json_dict['statistics']['subscriberCount']!='0'):
                if 'keywords' in json_dict['brandingSettings']['channel']:
                    try:
                        for kw in shlex.split(json_dict['brandingSettings']['channel']['keywords']):
                            kw_id = kw.lower().replace("'", "").replace("´", "").replace("’", "")
                            single_keys=nltk.word_tokenize(kw_id)
                            for key in single_keys:
                                if key not in all_keys:
                                    all_keys.append(key)
                                    g.add_node(key, type='keyword')

                    except:
                        for kw in json_dict['brandingSettings']['channel']['keywords'].split():
                            kw_id = kw.lower().replace("'", "").replace("´", "").replace("’", "")
                            for key in single_keys:
                                if key not in all_keys:
                                    all_keys.append(key)
                                    g.add_node(key, type='keyword')
                
                edges= itertools.combinations(all_keys,2)
                g.add_edges_from(edges)
                            

In [36]:
 nx.write_gexf(g, "test.gexf")